In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
def get_current_time(*args, **kwargs):
  """Returns the current time in H:MM AM/PM format"""
  import datetime

  now = datetime.datetime.now()
  return now.strftime("%I:%M %p")

In [ ]:
%pip install wikipedia

In [4]:
def search_wikipedia(query):
  """Searches Wikipedia and returns the summary of the first result"""
  from wikipedia import summary

  try:
    return summary(query, sentences=2)
  except:
    return "I couldn't find any information on that."

In [7]:
from langchain_core.tools import Tool

tools = [
  Tool(
    name="Time",
    func=get_current_time,
    description="Useful for when you need to know the current time."
  ),
  Tool(
    name="Wikipedia",
    func=search_wikipedia,
    description="Useful for when you need to search information about a topic."
  ),
]

In [9]:
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
from langchain import hub
prompt = hub.pull("hwchase17/structured-chat-agent:ea510f70")

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0,
)

In [13]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [14]:
from langchain.agents import create_structured_chat_agent

agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

In [15]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor.from_agent_and_tools(
  agent=agent,
  tools=tools,
  verbose=True,
  memory=memory,  # use the conversation memory to maintain context
  handle_parsing_errors=True,
)

In [16]:
initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: Time and Wikipedia."

In [19]:
from langchain_core.messages import SystemMessage

memory.chat_memory.add_message(SystemMessage(content=initial_message))

In [20]:
from langchain_core.messages import HumanMessage, AIMessage

while True:
  user_input = input("User: ")
  if user_input.lower() == "exit":
    break

  # adding the user's input message to the conversation memory
  memory.chat_memory.add_message(HumanMessage(content=user_input))

  # Invoke the agent with the user input and the current chat history
  response = agent_executor.invoke({"input": user_input})
  print("Bot: ", response["output"])

  # addint the agent's response to the conversation memory
  memory.chat_memory.add_message(AIMessage(content=response['output']))



> Entering new AgentExecutor chain...
Thought: I need to find out who the current president of Brazil is and their age. I will use Wikipedia to get the most accurate and up-to-date information.

Action:
```
{
  "action": "Wikipedia",
  "action_input": "current president of Brazil"
}
```The president of Brazil (Portuguese: presidente do Brasil), officially the president of the Federative Republic of Brazil (Portuguese: presidente da República Federativa do Brasil) or simply the President of the Republic, is the head of state and head of government of Brazil. The president leads the executive branch of the federal government and is the commander-in-chief of the Brazilian Armed Forces.Action:
```
{
  "action": "Wikipedia",
  "action_input": "current president of Brazil 2023"
}
```The president of Brazil (Portuguese: presidente do Brasil), officially the president of the Federative Republic of Brazil (Portuguese: presidente da República Federativa do Brasil) or simply the President of th